**Problem Statement**

Perform sentimental analysis on the Elon-musk tweets (Exlon-musk.csv)

In [29]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import string
import scipy #Language models library
import re #Regular Expression

from nltk.corpus import stopwords
from wordcloud import WordCloud

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [30]:
from google.colab import files
uploaded=files.upload()

Saving Elon_musk.csv to Elon_musk (1).csv


In [31]:
elon=pd.read_csv("Elon_musk.csv",encoding='ISO-8859-1')
elon.drop('Unnamed: 0',axis=1,inplace=True)
elon

,Text
0,@kunalb11 Im an alien
1,@ID_AA_Carmack Ray tracing on Cyberpunk with H...
2,@joerogan @Spotify Great interview!
3,@gtera27 Doge is underestimated
4,@teslacn Congratulations Tesla China for amazi...
...,...
1994,"@flcnhvy True, it sounds so surreal, but the n..."
1995,@PPathole Make sure to read ur terms &amp; con...
1996,@TeslaGong @PPathole Samwise Gamgee
1997,@PPathole Altho Dumb and Dumber is <U+0001F525...


**Data Exploration**

In [32]:
#No. of words in each tweet
elon['word_count']=elon['Text'].apply(lambda x: len(str(x).split(' ')))
elon.head(10)

,Text,word_count
0,@kunalb11 Im an alien,4
1,@ID_AA_Carmack Ray tracing on Cyberpunk with H...,13
2,@joerogan @Spotify Great interview!,4
3,@gtera27 Doge is underestimated,4
4,@teslacn Congratulations Tesla China for amazi...,17
5,Happy New Year of the Ox! https://t.co/9WFKMYu2oj,7
6,"Frodo was the underdoge,\nAll thought he would...",12
7,@OwenSparks_ @flcnhvy @anonyx10 Haha thanks :),6
8,@flcnhvy @anonyx10 Indeed! Tweets definitely d...,11
9,The most entertaining outcome is the most likely,8


In [33]:
#No. of Stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:
stop = stopwords.words('english')
elon['stopword_count']=elon['Text'].apply(lambda x: len([x for x in x.split() if x in stop])) 
elon.head(10)

,Text,word_count,stopword_count
0,@kunalb11 Im an alien,4,1
1,@ID_AA_Carmack Ray tracing on Cyberpunk with H...,13,4
2,@joerogan @Spotify Great interview!,4,0
3,@gtera27 Doge is underestimated,4,1
4,@teslacn Congratulations Tesla China for amazi...,17,5
5,Happy New Year of the Ox! https://t.co/9WFKMYu2oj,7,2
6,"Frodo was the underdoge,\nAll thought he would...",12,5
7,@OwenSparks_ @flcnhvy @anonyx10 Haha thanks :),6,0
8,@flcnhvy @anonyx10 Indeed! Tweets definitely d...,11,2
9,The most entertaining outcome is the most likely,8,4


In [35]:
#No. of digits
elon['digits'] = elon['Text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
elon.head()

,Text,word_count,stopword_count,digits
0,@kunalb11 Im an alien,4,1,0
1,@ID_AA_Carmack Ray tracing on Cyberpunk with H...,13,4,0
2,@joerogan @Spotify Great interview!,4,0,0
3,@gtera27 Doge is underestimated,4,1,0
4,@teslacn Congratulations Tesla China for amazi...,17,5,0


In [36]:
elon.digits.value_counts()

0     1854
1      117
2       23
3        3
4        1
10       1
Name: digits, dtype: int64

In [40]:
#No. of URL's in the data
elon['web_links'] = elon['Text'].apply(lambda x: len([x for x in x.split() if x.startswith('https://')]))
elon.head(10)

,Text,word_count,stopword_count,digits,web_links,hashtag
0,@kunalb11 Im an alien,4,1,0,0,0
1,@ID_AA_Carmack Ray tracing on Cyberpunk with H...,13,4,0,0,0
2,@joerogan @Spotify Great interview!,4,0,0,0,0
3,@gtera27 Doge is underestimated,4,1,0,0,0
4,@teslacn Congratulations Tesla China for amazi...,17,5,0,0,0
5,Happy New Year of the Ox! https://t.co/9WFKMYu2oj,7,2,0,1,0
6,"Frodo was the underdoge,\nAll thought he would...",12,5,0,1,0
7,@OwenSparks_ @flcnhvy @anonyx10 Haha thanks :),6,0,0,0,0
8,@flcnhvy @anonyx10 Indeed! Tweets definitely d...,11,2,0,0,0
9,The most entertaining outcome is the most likely,8,4,0,0,0


In [38]:
elon.web_links.value_counts()

0    1356
1     632
2      11
Name: web_links, dtype: int64

In [39]:
#No. of Hastags in the data
elon['hashtag'] = elon['Text'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
elon.head(10)

,Text,word_count,stopword_count,digits,web_links,hashtag
0,@kunalb11 Im an alien,4,1,0,0,0
1,@ID_AA_Carmack Ray tracing on Cyberpunk with H...,13,4,0,0,0
2,@joerogan @Spotify Great interview!,4,0,0,0,0
3,@gtera27 Doge is underestimated,4,1,0,0,0
4,@teslacn Congratulations Tesla China for amazi...,17,5,0,0,0
5,Happy New Year of the Ox! https://t.co/9WFKMYu2oj,7,2,0,1,0
6,"Frodo was the underdoge,\nAll thought he would...",12,5,0,1,0
7,@OwenSparks_ @flcnhvy @anonyx10 Haha thanks :),6,0,0,0,0
8,@flcnhvy @anonyx10 Indeed! Tweets definitely d...,11,2,0,0,0
9,The most entertaining outcome is the most likely,8,4,0,0,0


In [41]:
elon.hashtag.value_counts()

0    1990
1       5
2       3
4       1
Name: hashtag, dtype: int64

**Text Processing**

In [42]:
#Collecting all the tweets
elon=[Text.strip() for Text in elon.Text]
elon=[Text for Text in elon if Text] #Empty string removal
elon[0:5]

['@kunalb11 I\x92m an alien',
 '@ID_AA_Carmack Ray tracing on Cyberpunk with HDR is next-level. Have you tried it?',
 '@joerogan @Spotify Great interview!',
 '@gtera27 Doge is underestimated',
 '@teslacn Congratulations Tesla China for amazing execution last year. Now on to the next for even more!!']

In [43]:
#Now joining all tweets into single text
elon_tweets=" ".join(elon)
elon_tweets

'@kunalb11 I\x92m an alien @ID_AA_Carmack Ray tracing on Cyberpunk with HDR is next-level. Have you tried it? @joerogan @Spotify Great interview! @gtera27 Doge is underestimated @teslacn Congratulations Tesla China for amazing execution last year. Now on to the next for even more!! Happy New Year of the Ox! https://t.co/9WFKMYu2oj Frodo was the underdoge,\nAll thought he would fail,\nHimself most of all. https://t.co/zGxJFDzzrM @OwenSparks_ @flcnhvy @anonyx10 Haha thanks :) @flcnhvy @anonyx10 Indeed! Tweets definitely do not represent real-world time allocation. The most entertaining outcome is the most likely @GiveDirectly Just sent some Just agree to do Clubhouse with @kanyewest https://t.co/3rWE9uHSTS @geoffkeighley @UnrealEngine It\x92s getting real Bought some Dogecoin for lil X, so he can be a toddler hodler @JoshManMode He definitely has issues, but the sentencing seems a bit high @freewalletorg Thanks for fixing @freewalletorg Please unlock my account @AstroJordy <U+0001F923><U